In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics
import talib as ta
from finta import TA

import warnings
warnings.filterwarnings('ignore')

In [13]:
# the pandas webreader provides remote data access to apis
import pandas_datareader as webreader

date_today = "2022-11-14"
#AKA as far back as fear greed index will go
date_start = "2016-1-1"

# set the symbol to bitcoin-usd quotes
# for more symbols check yahoo.finance.com
symbol = "BTC-USD"

In [14]:
# now we will send the request to the yahoo finance api endpoint
df = webreader.DataReader(symbol, start=date_start, end=date_today, data_source="yahoo")
df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-01,436.246002,427.515015,430.721008,434.334015,36278900,434.334015
2016-01-02,436.062012,431.869995,434.622009,433.437988,30096600,433.437988
2016-01-03,433.743011,424.705994,433.578003,430.010986,39633800,430.010986
2016-01-04,434.516998,429.084015,430.061005,433.091003,38477500,433.091003
2016-01-05,434.182007,429.675995,433.069000,431.959991,34522600,431.959991


In [15]:
df['50 SMA'] = ta.SMA(df['Close'], timeperiod=50)
df['200 SMA'] = ta.SMA(df['Close'], timeperiod=200)
df['10 EMA'] = ta.EMA(df['Close'], timeperiod=10)
df['20 EMA'] = ta.EMA(df['Close'], timeperiod=20)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

In [16]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,50 SMA,200 SMA,10 EMA,20 EMA,RSI
Date,,,,,,,,,,,
2022-11-10,18054.312500,15834.018555,15883.158203,17586.771484,83202283721,17586.771484,19584.083730,23547.594272,19118.173558,19554.702520,38.131300
2022-11-11,17650.943359,16543.482422,17583.251953,17034.292969,55871616488,17034.292969,19536.498574,23430.474194,18739.286178,19314.663515,35.836612
2022-11-12,17066.675781,16651.775391,17036.875000,16799.185547,29717699419,16799.185547,19486.529512,23323.882817,18386.540609,19075.094185,34.874802
2022-11-13,16920.765625,16320.634766,16799.722656,16353.365234,27209183682,16353.365234,19434.856582,23209.444038,18016.872359,18815.881904,33.062715
2022-11-14,17095.193359,15876.135742,16363.816406,16415.470703,48733945856,16415.470703,19387.124043,23092.652251,17725.708422,18587.271313,33.580462


In [17]:
FeerGreed = pd.read_csv("Feer_greed.csv")  
FeerGreed




,Date,value,value_classification
0,2022-11-14,24,Extreme Fear
1,2022-11-13,22,Extreme Fear
2,2022-11-12,21,Extreme Fear
3,2022-11-11,25,Extreme Fear
4,2022-11-10,22,Extreme Fear
...,...,...,...
1740,2018-02-05,11,Extreme Fear
1741,2018-02-04,24,Extreme Fear
1742,2018-02-03,40,Fear
1743,2018-02-02,15,Extreme Fear


In [18]:

FeerGreed.set_index('Date',inplace=True)
FeerGreed

,value,value_classification
Date,,
2022-11-14,24,Extreme Fear
2022-11-13,22,Extreme Fear
2022-11-12,21,Extreme Fear
2022-11-11,25,Extreme Fear
2022-11-10,22,Extreme Fear
...,...,...
2018-02-05,11,Extreme Fear
2018-02-04,24,Extreme Fear
2018-02-03,40,Fear


In [19]:
FeerGreed.index = pd.to_datetime(FeerGreed.index)

In [20]:
dfmerged = df.merge(FeerGreed,on='Date')
dfmerged.head()


,High,Low,Open,Close,Volume,Adj Close,50 SMA,200 SMA,10 EMA,20 EMA,RSI,value,value_classification
Date,,,,,,,,,,,,,
2018-02-01,10288.799805,8812.280273,10237.299805,9170.540039,9959400448,9170.540039,14036.866816,7856.521897,10769.754729,11614.719069,32.505630,30,Fear
2018-02-02,9142.280273,7796.490234,9142.280273,8830.750000,12726899712,8830.750000,13882.201816,7889.533597,10417.208414,11349.579157,31.214760,15,Extreme Fear
2018-02-03,9430.750000,8251.629883,8852.120117,9174.910156,7263790080,9174.910156,13711.562012,7923.813749,10191.336004,11142.467824,34.070623,40,Fear
2018-02-04,9334.870117,8031.220215,9175.700195,8277.009766,7073549824,8277.009766,13487.154199,7953.831648,9843.276688,10869.567056,30.511391,24,Extreme Fear
2018-02-05,8364.839844,6756.680176,8270.540039,6955.270020,9285289984,6955.270020,13243.443584,7974.519998,9318.184566,10496.776862,26.176396,11,Extreme Fear


In [29]:
dfmerged.to_csv('BTC_USD.csv')